#### Importation des librairies :

In [19]:
pip install xgboost


  Using cached xgboost-2.1.0-py3-none-win_amd64.whl.metadata (2.1 kB)
Using cached xgboost-2.1.0-py3-none-win_amd64.whl (124.9 MB)


In [4]:
pip install mlflow

Note: you may need to restart the kernel to use updated packages.


In [20]:
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import gc
import mlflow
import mlflow.sklearn
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
import xgboost as xgb

In [7]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

#### Importation des fichiers : 

In [3]:
# Chemin vers le dossier contenant les fichiers extraits
dossier_data = "C:/Users/paulm/Documents/GitHub/Projet7/data"

# Liste des fichiers dans le dossier data
fichiers_data = os.listdir(dossier_data)
print(f"Fichiers dans le dossier data : {fichiers_data}")

chemin_fichier_train = os.path.join(dossier_data, 'application_test.csv')
train_data = pd.read_csv(chemin_fichier_train)

# Affichage des premières lignes du DataFrame pour vérifier que le fichier a été chargé correctement
print(train_data.head())

Fichiers dans le dossier data : ['application_test.csv', 'application_train.csv', 'bureau.csv', 'bureau_balance.csv', 'credit_card_balance.csv', 'HomeCredit_columns_description.csv', 'installments_payments.csv', 'POS_CASH_balance.csv', 'previous_application.csv', 'processed_data.csv', 'Projet+Mise+en+prod+-+home-credit-default-risk (1).zip', 'sample_submission.csv']
   SK_ID_CURR NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  \
0      100001         Cash loans           F            N               Y   
1      100005         Cash loans           M            N               Y   
2      100013         Cash loans           M            Y               Y   
3      100028         Cash loans           F            N               Y   
4      100038         Cash loans           M            Y               N   

   CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  AMT_GOODS_PRICE  \
0             0          135000.0    568800.0      20560.5         450000.0   
1         

In [8]:
# Chemin vers le dossier où se trouve le fichier processed_data.csv
chemin_dossier = "C:/Users/paulm/Documents/GitHub/Projet7/data/"

# Charger les données
df = pd.read_csv(os.path.join(chemin_dossier, 'processed_data.csv'))

# Vérifier les données chargées
print(df.head())

   SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
0      100002       1         Cash loans           M            N   
1      100003       0         Cash loans           F            N   
2      100004       0    Revolving loans           M            Y   
3      100006       0         Cash loans           F            N   
4      100007       0         Cash loans           M            N   

  FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
0               Y             0          202500.0    406597.5      24700.5   
1               N             0          270000.0   1293502.5      35698.5   
2               Y             0           67500.0    135000.0       6750.0   
3               Y             0          135000.0    312682.5      29686.5   
4               Y             0          121500.0    513000.0      21865.5   

   ...  CC_CNT_INSTALMENT_MATURE_CUM_MEAN CC_CNT_INSTALMENT_MATURE_CUM_SUM  \
0  ...                                

#### Initialisation de l'environnement MLFlow :

## Préparation des données et feature engineering :

### Charger les données prétraitées et entraîner le modèle

In [10]:
# Function to load preprocessed data from disk
def load_from_disk(file_name):
    return pd.read_csv(file_name)

# Handling infinity values
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop non-numeric columns
non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns
df.drop(columns=non_numeric_cols, inplace=True)
 
# Features and labels
train_features = df.drop(columns=['TARGET'])
train_labels = df['TARGET']

# Handling missing values using SimpleImputer
imputer = SimpleImputer(strategy='mean')
train_features = imputer.fit_transform(train_features)

# Normalization
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)

# Model training and evaluation
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=47)
oof_preds = np.zeros(train_features.shape[0])

# Start MLflow run
with mlflow.start_run():
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_features, train_labels)):
        train_x, train_y = train_features[train_idx], train_labels[train_idx]
        valid_x, valid_y = train_features[valid_idx], train_labels[valid_idx]

        model = LogisticRegression(C=0.0001, class_weight='balanced', solver='lbfgs', max_iter=2000)
        model.fit(train_x, train_y)
        oof_preds[valid_idx] = model.predict_proba(valid_x)[:, 1]

        fold_auc = roc_auc_score(valid_y, oof_preds[valid_idx])
        print(f"Fold {n_fold + 1} AUC: {fold_auc}")

        # Log fold AUC score to MLflow
        mlflow.log_metric(f"Fold {n_fold + 1} AUC", fold_auc)

    # Calculate overall AUC score
    score = roc_auc_score(train_labels, oof_preds)
    print(f"Full AUC score: {score}")

    # Log parameters and metrics to MLflow
    mlflow.log_param("C", 0.0001)
    mlflow.log_param("class_weight", 'balanced')
    mlflow.log_param("solver", 'lbfgs')
    mlflow.log_param("max_iter", 2000)
    mlflow.log_metric("Full AUC score", score)

    # Log the model to MLflow
    mlflow.sklearn.log_model(model, "model")

Fold 1 AUC: 0.757367759893559
Fold 2 AUC: 0.7583632111017994
Fold 3 AUC: 0.7614197377646141
Fold 4 AUC: 0.7591545829918508
Fold 5 AUC: 0.7554244196347567
Full AUC score: 0.7583332894798088


### Utilisation d'autres algorythmes de classification :

#### Random forest

Avec mlflow

In [14]:
# Parameters for Random Forest
params = {
    'n_estimators': 1000,
    'max_depth': 10,
    'min_samples_split': 5,
    'min_samples_leaf': 2,
    'max_features': 'sqrt',
    'bootstrap': True,
    'n_jobs': -1,
    'random_state': 47
}

# Model training and evaluation
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=47)
oof_preds = np.zeros(train_features.shape[0])

# Start MLflow run
with mlflow.start_run():
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_features, train_labels)):
        train_x, train_y = train_features[train_idx], train_labels[train_idx]
        valid_x, valid_y = train_features[valid_idx], train_labels[valid_idx]

        clf = RandomForestClassifier(**params)
        clf.fit(train_x, train_y)

        oof_preds[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        fold_auc = roc_auc_score(valid_y, oof_preds[valid_idx])
        print(f"Fold {n_fold + 1} AUC: {fold_auc}")

        # Log fold AUC score to MLflow
        mlflow.log_metric(f"Fold {n_fold + 1} AUC", fold_auc)

    # Calculate overall AUC score
    score = roc_auc_score(train_labels, oof_preds)
    print(f"Full AUC score: {score}")

    # Log parameters and metrics to MLflow
    mlflow.log_params(params)
    mlflow.log_metric("Full AUC score", score)

    # Log the model to MLflow
    mlflow.sklearn.log_model(clf, "model")

Fold 1 AUC: 0.7428157775125228
Fold 2 AUC: 0.7439050055911661
Fold 3 AUC: 0.7513792400848425
Fold 4 AUC: 0.7467853907967583
Fold 5 AUC: 0.7455999449673726
Full AUC score: 0.7460842093546884


c:\Users\paulm\anaconda3\envs\notebook-6.5.7\Lib\site-packages\_distutils_hack\__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\paulm\anaconda3\envs\notebook-6.5.7\Lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


### Création d'un score métier :

Nous allons donc regarder la distribution pour comprendre un peu mieux :

#### Visualisation des classes

In [15]:
# Afficher les classes et leur distribution
class_distribution = df['TARGET'].value_counts()
print("Distribution des classes :")
print(class_distribution)

# Afficher le pourcentage de chaque classe
total_samples = df.shape[0]
print("\nPourcentage de chaque classe :")
for class_label, count in class_distribution.items():
    percentage = (count / total_samples) * 100
    print(f"Classe {class_label}: {percentage:.2f}%")

Distribution des classes :
TARGET
0    282686
1     24825
Name: count, dtype: int64

Pourcentage de chaque classe :
Classe 0: 91.93%
Classe 1: 8.07%


On observe en effet un déséquilibre significatif entre les classes, avec environ 91.93% d'étiquettes "0" et seulement 8.07% d'étiquettes "1".

Le score métier n'est toujours pas satisfaisant nous allons tenter une nouvelle approche : 

### Calcul du Score Métier
 
Le score métier sera calculé en fonction des prédictions du modèle et des coûts définis. Nous devons ajuster le seuil de décision pour maximiser ce score. Pour cela, nous allons :

Obtenir les probabilités prédites par le modèle.
Ajuster le seuil de décision pour minimiser le coût total des erreurs en tenant compte des poids FP et FN.

In [16]:
cost_fp = 1  # Coût d'un faux positif
cost_fn = 10  # Coût d'un faux négatif (10 fois le coût d'un FP)

In [21]:
# Suppression des colonnes non numériques
non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns
df.drop(columns=non_numeric_cols, inplace=True)

# Séparation des caractéristiques et des étiquettes
X = df.drop(columns=['TARGET'])
y = df['TARGET']

# Gestion des valeurs manquantes
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Normalisation
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialisation du modèle
model = RandomForestClassifier(random_state=42)

# Entraînement du modèle
model.fit(X_train, y_train)

# Obtenir les probabilités prédites par le modèle pour la classe 1
y_prob = model.predict_proba(X_test)[:, 1]

# Calculer les coûts pour chaque seuil de décision
thresholds = np.linspace(0, 1, 100)
costs = []
for threshold in thresholds:
    y_pred_thresholded = (y_prob > threshold).astype(int)
    fp = np.sum((y_pred_thresholded == 1) & (y_test == 0)) * cost_fp
    fn = np.sum((y_pred_thresholded == 0) & (y_test == 1)) * cost_fn
    total_cost = fp + fn
    costs.append(total_cost)

# Trouver le seuil qui minimise le coût total des erreurs
best_threshold = thresholds[np.argmin(costs)]
print(f"Meilleur seuil pour minimiser le coût métier : {best_threshold}")

# Utiliser le meilleur seuil pour prédire les classes
y_pred_best_threshold = (y_prob > best_threshold).astype(int)

# Afficher le rapport de classification avec le meilleur seuil
print("Rapport de classification avec le meilleur seuil :")
print(classification_report(y_test, y_pred_best_threshold))

# Afficher la matrice de confusion avec le meilleur seuil
print("Matrice de confusion avec le meilleur seuil :")
print(confusion_matrix(y_test, y_pred_best_threshold))

Meilleur seuil pour minimiser le coût métier : 0.11111111111111112
Rapport de classification avec le meilleur seuil :
              precision    recall  f1-score   support

           0       0.95      0.73      0.83     56554
           1       0.16      0.59      0.25      4949

    accuracy                           0.72     61503
   macro avg       0.56      0.66      0.54     61503
weighted avg       0.89      0.72      0.78     61503

Matrice de confusion avec le meilleur seuil :
[[41162 15392]
 [ 2029  2920]]


 **Classe 0 (Majoritaire)** :
  - Précision : 0.96 (Très bonne)
  - Rappel : 0.73 (Modéré)
  - F1-Score : 0.83 (Bon)
  
 **Classe 1 (Minoritaire)** :
  - Précision : 0.17 (Très faible)
  - Rappel : 0.62 (Modéré)
  - F1-Score : 0.26 (Faible)
- **Exactitude globale** : 0.72 (Modéré)


True Negatives (TN) : 41320
False Positives (FP) : 15234
False Negatives (FN) : 1876
True Positives (TP) : 3073

In [22]:
# Définir les modèles à tester avec leurs paramètres
models = [
    ('Dummy Classifier', DummyClassifier(strategy='most_frequent')),
    ('Random Forest', RandomForestClassifier(class_weight='balanced', random_state=42)),
    ('Logistic Regression', LogisticRegression(class_weight='balanced', random_state=42)),
    ('K-Nearest Neighbors', KNeighborsClassifier()),
    ('XGBoost', xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42))
]

# Définir les paramètres à tester pour chaque modèle (le DummyClassifier n'a pas de paramètres à ajuster)
params = {
    'Dummy Classifier': {},
    'Random Forest': {'n_estimators': [50, 100]},
    'Logistic Regression': {'C': [1.0, 10.0]},
    'K-Nearest Neighbors': {'n_neighbors': [5, 10]},
    'XGBoost': {'n_estimators': [50, 100], 'learning_rate': [0.1, 0.01], 'max_depth': [3, 5]}
}

# Configurer l'expérience MLflow
mlflow.set_experiment("Credit Scoring Experiment")

# Échantillonner les données
sample_size = 0.1  # Utiliser 10% des données pour un échantillonnage rapide
X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, test_size=(1 - sample_size), random_state=42)
X_test_sample, _, y_test_sample, _ = train_test_split(X_test, y_test, test_size=(1 - sample_size), random_state=42)

# Tester les modèles et trouver les meilleurs paramètres
for name, model in models:
    with mlflow.start_run(run_name=name):
        print(f"Entraînement du modèle {name}...")
        
        if name == 'Dummy Classifier':
            model.fit(X_train_sample, y_train_sample)
            best_model = model
            y_pred = model.predict(X_test_sample)
        else:
            grid_search = GridSearchCV(estimator=model, param_grid=params[name], scoring='f1', cv=3)
            grid_search.fit(X_train_sample, y_train_sample)
            
            # Loguer les paramètres dans MLflow
            mlflow.log_params(grid_search.best_params_)
            
            # Afficher les meilleurs paramètres et le score F1
            print(f"Meilleurs paramètres pour {name}: {grid_search.best_params_}")
            print(f"Score F1 moyen sur le jeu de validation: {grid_search.best_score_:.3f}")
            
            # Utiliser le meilleur modèle pour les prédictions
            best_model = grid_search.best_estimator_
            y_pred = best_model.predict(X_test_sample)
        
        # Évaluer le modèle avec le meilleur seuil pour minimiser le coût métier
        y_prob = best_model.predict_proba(X_test_sample)[:, 1]
        thresholds = np.linspace(0, 1, 100)
        costs = []
        for threshold in thresholds:
            y_pred_thresholded = (y_prob > threshold).astype(int)
            fp = np.sum((y_pred_thresholded == 1) & (y_test_sample == 0)) * cost_fp
            fn = np.sum((y_pred_thresholded == 0) & (y_test_sample == 1)) * cost_fn
            total_cost = fp + fn
            costs.append(total_cost)
        
        best_threshold = thresholds[np.argmin(costs)]
        print(f"Meilleur seuil pour minimiser le coût métier avec {name}: {best_threshold}")
        
        y_pred_best_threshold = (y_prob > best_threshold).astype(int)
        
        # Afficher le rapport de classification et la matrice de confusion
        classification_report_str = classification_report(y_test_sample, y_pred_best_threshold)
        confusion_matrix_str = confusion_matrix(y_test_sample, y_pred_best_threshold)
        
        print(f"Rapport de classification avec le meilleur seuil pour {name}:")
        print(classification_report_str)
        
        print(f"Matrice de confusion avec le meilleur seuil pour {name}:")
        print(confusion_matrix_str)
        print()
        
        # Loguer les métriques et les résultats dans MLflow
        mlflow.log_metric("best_score_f1", grid_search.best_score_ if name != 'Dummy Classifier' else 0)
        mlflow.log_metric("best_threshold", best_threshold)
        mlflow.log_text(classification_report_str, "classification_report.txt")
        mlflow.log_text(str(confusion_matrix_str), "confusion_matrix.txt")
        
        # Loguer le modèle
        mlflow.sklearn.log_model(best_model, "model")

Entraînement du modèle Dummy Classifier...
Meilleur seuil pour minimiser le coût métier avec Dummy Classifier: 0.0
Rapport de classification avec le meilleur seuil pour Dummy Classifier:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      5664
           1       0.00      0.00      0.00       486

    accuracy                           0.92      6150
   macro avg       0.46      0.50      0.48      6150
weighted avg       0.85      0.92      0.88      6150

Matrice de confusion avec le meilleur seuil pour Dummy Classifier:
[[5664    0]
 [ 486    0]]



c:\Users\paulm\anaconda3\envs\notebook-6.5.7\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\paulm\anaconda3\envs\notebook-6.5.7\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\paulm\anaconda3\envs\notebook-6.5.7\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

Entraînement du modèle Random Forest...
Meilleurs paramètres pour Random Forest: {'n_estimators': 50}
Score F1 moyen sur le jeu de validation: 0.001
Meilleur seuil pour minimiser le coût métier avec Random Forest: 0.08080808080808081
Rapport de classification avec le meilleur seuil pour Random Forest:
              precision    recall  f1-score   support

           0       0.95      0.70      0.80      5664
           1       0.14      0.56      0.22       486

    accuracy                           0.69      6150
   macro avg       0.54      0.63      0.51      6150
weighted avg       0.88      0.69      0.76      6150

Matrice de confusion avec le meilleur seuil pour Random Forest:
[[3950 1714]
 [ 212  274]]



c:\Users\paulm\anaconda3\envs\notebook-6.5.7\Lib\site-packages\_distutils_hack\__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\paulm\anaconda3\envs\notebook-6.5.7\Lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Entraînement du modèle Logistic Regression...


...

c:\Users\paulm\anaconda3\envs\notebook-6.5.7\Lib\site-packages\_distutils_hack\__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\paulm\anaconda3\envs\notebook-6.5.7\Lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Entraînement du modèle XGBoost...


c:\Users\paulm\anaconda3\envs\notebook-6.5.7\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:29:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\paulm\anaconda3\envs\notebook-6.5.7\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:29:25] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\paulm\anaconda3\envs\notebook-6.5.7\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:29:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(sm

Meilleurs paramètres pour XGBoost: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
Score F1 moyen sur le jeu de validation: 0.055
Meilleur seuil pour minimiser le coût métier avec XGBoost: 0.08080808080808081
Rapport de classification avec le meilleur seuil pour XGBoost:
              precision    recall  f1-score   support

           0       0.96      0.74      0.83      5664
           1       0.17      0.62      0.26       486

    accuracy                           0.73      6150
   macro avg       0.56      0.68      0.55      6150
weighted avg       0.89      0.73      0.79      6150

Matrice de confusion avec le meilleur seuil pour XGBoost:
[[4185 1479]
 [ 187  299]]



c:\Users\paulm\anaconda3\envs\notebook-6.5.7\Lib\site-packages\_distutils_hack\__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\paulm\anaconda3\envs\notebook-6.5.7\Lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Conclusion générale:

- Dummy Classifier sert de baseline et ne détecte aucune instance de la classe 1.
- Random Forest et Logistic Regression ont des performances intermédiaires, avec Logistic Regression offrant un meilleur - compromis entre précision et recall pour la classe 1.
- KNN a des performances médiocres pour la classe 1.
- XGBoost est le meilleur modèle parmi ceux testés pour détecter la classe 1, avec un bon compromis entre précision et recall, rendant ses résultats plus équilibrés et utilisables en pratique.


En utilisant le score métier, nous avons pu ajuster le modèle pour minimiser le coût associé aux erreurs de classification, en particulier en accordant plus d'importance à la réduction des faux négatifs (FN). Cela permet de répondre aux exigences spécifiques du projet où le coût d'un FN est considérablement plus élevé que celui d'un FP.